In [195]:
import torch
from botorch.models import SingleTaskGP, MixedSingleTaskGP, SaasFullyBayesianSingleTaskGP, ModelListGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import LogExpectedImprovement, UpperConfidenceBound, ProbabilityOfImprovement
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf
from botorch import fit_fully_bayesian_model_nuts
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import matplotlib.animation as animation
from scipy.stats import multivariate_normal
from botorch.utils.multi_objective.box_decompositions.dominated import DominatedPartitioning
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.acquisition.multi_objective.logei import qLogNoisyExpectedHypervolumeImprovement
from botorch.acquisition.multi_objective.analytic import ExpectedHypervolumeImprovement
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning

In [ ]:
X = torch.rand(20, 2)
Y = torch.stack([torch.sin(X[:, 0]), torch.cos(X[:, 1])], -1)
cat_dims = [0]

gp = MixedSingleTaskGP(X, Y,
    cat_dims=cat_dims,
    cont_kernel_factory=None)

gp = SingleTaskGP(X, Y,
    covar_module=None)

models_list = []
for i in range(Y.shape[1]):
    model = SaasFullyBayesianSingleTaskGP(X, Y[:,i].reshape(-1,1))
    fit_fully_bayesian_model_nuts(model, num_samples=1, disable_progbar=True)
    models_list.append(model)
models_list = tuple(models_list)
#gp = ModelListGP(*models_list)
#gp.posterior(X).mean can't be used we do the following.

C:\Users\danin\AppData\Local\Temp\ipykernel_17828\191361246.py:9: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  gp = SingleTaskGP(X, Y,
C:\Users\danin\AppData\Local\Temp\ipykernel_17828\191361246.py:14: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SaasFullyBayesianSingleTaskGP(X, Y[:,i].reshape(-1,1))
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:267: InputDataWarning: Data (outcome observations) is not standardized (std = tensor([0.2497]), mean = tensor([0.4990])).Please consider scali

In [ ]:
ref_point = [0.0, 0.0]

# Partitioning from current Y
partitioning = DominatedPartitioning(
    ref_point=torch.tensor(ref_point),
    Y=Y
)

# Create EHVI acquisition
ehvi = ExpectedHypervolumeImprovement(
    model=gp,
    ref_point=ref_point,
    partitioning=partitioning
)

candidate, _ = optimize_acqf(ehvi,q=1,num_restarts=10, raw_samples=1024, bounds=torch.tensor([[0.0, 0.0], [1.0, 1.0]]))

In [212]:
Y_new = torch.cat((Y,prediction))
np.shape(Y), np.shape(Y_new), np.shape(prediction)
X_new = torch.cat((X, candidate))
np.shape(X), np.shape(X_new), np.shape(candidate)

(torch.Size([20, 2]), torch.Size([21, 2]), torch.Size([1, 2]))

In [198]:
candidate, _

(tensor([[0.8830, 0.2889]]), tensor(0.7093))

In [202]:
prediction = gp.posterior(candidate).mean
prediction

tensor([[0.7522, 0.9486]], grad_fn=<TransposeBackward0>)

In [28]:
def poly(a, b, c, *args, ka=0, kb=0, kc=0, **kwargs):
    print(a, b, c)
    print(repr(args))
    print(ka, kb, kc)
    print(repr(kwargs))

In [29]:
dictt = {'a':[5],'b':[5]}
key = 'a'
poly(0, 1, 2, 4, 5, 6, kb=7, ua=8, ub=9)

0 1 2
(4, 5, 6)
0 7 0
{'ua': 8, 'ub': 9}
